In [1]:
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
import os
import torch
from tqdm import tqdm
import numpy as np
import json
import gc
from torchvision.models import DenseNet121_Weights
from torchvision import transforms
from torchvision import models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from diffusers.utils import load_image

In [26]:
categories = [
    # 'Automobile',
    # 'Beverages',
    # 'Clothing',
    # 'Electronics',
    # 'Entertainment',
    # 'Food',
    # 'Jewellery',
    # 'Makeup',
    # 'Non-profit',
    'Shoes'
]

quadrant = [
    'HF-HE',
    'HF-LE',
    # 'LF-HE',
    # 'LF-LE'
]

for category in categories:
    for quad in quadrant:
        folder_path = f'D://Study//Project//Major project//Hashtag analysis//{category}//{quad}'

        weights = DenseNet121_Weights.IMAGENET1K_V1
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = models.densenet121(weights=weights).to(device)
        model.eval()

        preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
        ])

        output_dict = {}
        for img in tqdm(os.listdir(folder_path)):
            image_name = img
            img = load_image(os.path.join(folder_path, img))
            with torch.no_grad():
                img = preprocess(img).unsqueeze(0).to(device)
                output_dict[image_name] = model(img).detach().cpu().numpy().tolist()
            del img 
            del image_name
            gc.collect()
        
        with open(f'D:\Study\Project\Major project\Hashtag analysis\{category}\{quad}_image_embeddings.json', 'w') as json_file:
            json.dump(output_dict, json_file, indent=4)

100%|██████████| 252/252 [00:57<00:00,  4.37it/s]


In [21]:
with open('HF-HE_image_embeddings.json', 'w') as json_file:
    json.dump(output_dict, json_file, indent=4)